<a href="https://colab.research.google.com/github/harelm4/Embible/blob/master/notebooks/distilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 15.2 MB/s 
     |████████████████████████████████| 212 kB 68.3 MB/s 
     |████████████████████████████████| 182 kB 68.3 MB/s 
     |████████████████████████████████| 132 kB 73.5 MB/s 
     |████████████████████████████████| 127 kB 69.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 11.0 MB/s 
     |████████████████████████████████| 7.6 MB 44.0 MB/s 


In [2]:
from transformers import AutoTokenizer,AutoModelForMaskedLM
cp = "distilbert-base-multilingual-cased"
model=AutoModelForMaskedLM.from_pretrained(cp)
tokenizer=AutoTokenizer.from_pretrained(cp)

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/542M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [3]:
import datasets
ds=datasets.load_dataset("csv", data_files='/content/drive/Shareddrives/Embible/Data/train_df.csv',sep="\t")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-79c7ec62ce633f86/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
def tokenize_function(dataset):
  return tokenizer(str(dataset["verse"])
                      )

tokenized_ds=ds.map(tokenize_function)

  0%|          | 0/22150 [00:00<?, ?ex/s]

In [5]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [6]:
from transformers import TrainingArguments


batch_size = 64
# Show the training loss with every epoch
logging_steps = len(ds["train"])

training_args = TrainingArguments(
    output_dir="/content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    logging_steps=logging_steps,

)


In [7]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [8]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [9]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: verse_idx, verse, Unnamed: 0, name. If verse_idx, verse, Unnamed: 0, name are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 22150
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1041
  Number of trainable parameters = 135445755
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using

Epoch,Training Loss,Validation Loss
1,No log,0.714159
2,No log,0.618226
3,No log,0.568545


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: verse_idx, verse, Unnamed: 0, name. If verse_idx, verse, Unnamed: 0, name are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 22150
  Batch size = 64
Saving model checkpoint to /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/checkpoint-500
Configuration saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/checkpoint-500/config.json
Model weights saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-fi

TrainOutput(global_step=1041, training_loss=0.6944689132294669, metrics={'train_runtime': 1211.7594, 'train_samples_per_second': 54.838, 'train_steps_per_second': 0.859, 'total_flos': 857658793307292.0, 'train_loss': 0.6944689132294669, 'epoch': 3.0})

In [10]:
trainer.save_model("/content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert")
model.save_pretrained("/content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert")

Saving model checkpoint to /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert
Configuration saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/config.json
Model weights saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/special_tokens_map.json
Configuration saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/config.json
Model weights saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/pytorch_model.bin
